<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/vietnamese_frequency_sentence_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget "https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar"

--2021-12-22 04:08:22--  https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar [following]
--2021-12-22 04:08:22--  https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7310993 (7.0M) [application/octet-stream]
Saving to: ‘EVBCorpus_EVBNews_v2.0.rar.2’

EVBCorpus_EVBNews_v 100%[===================>]   6.97M  --.-KB/s    in 0.08s   

2021-12-22 04:08:22 (84.8 MB/s) - ‘EVBCorpus_EVBNews_v2.0.rar.

In [ ]:
!unrar e "EVBCorpus_EVBNews_v2.0.rar"  &> /dev/null 

In [1]:
!pip install pandas==1.3.0
from pathlib import Path
import pandas as pd
pd.__version__

In [ ]:
pip install sgmllib3k
import sgmllib
class FoundTitle(Exception):
    pass

class ExtractTitle(sgmllib.SGMLParser):

    def __init__(self, verbose=0):
        sgmllib.SGMLParser.__init__(self, verbose)
        self.title = self.data = None

    def handle_data(self, data):
        if self.data is not None:
            self.data.append(data)

    def start_title(self, attrs):
        self.data = []

    def end_title(self):
        self.title = "".join(self.data)
        raise FoundTitle # abort parsing!

def extract(file):
    # extract title from an HTML/SGML stream
    p = ExtractTitle()
    try:
        while 1:
            # read small chunks
            s = file.read(512)
            if not s:
                break
            p.feed(s)
        p.close()
    except FoundTitle:
        return p.title
    return None
    
extract(open("N0001.sgml"))

In [3]:
from bs4 import BeautifulSoup
import re

def remove_punc(instring):
  outstring = re.sub(r'[^\w\s]', '', instring)
  return outstring  

sgml_files = Path('.').glob('*.sgml')
en = []
vn = []
for file in sgml_files:
  with open(file,'r') as f:
    doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    out = soup.find_all('spair')
    for i in range(len(out)):
      e = remove_punc(out[i].find('s', {'id':f'en{i+1}'}).getText())
      v = remove_punc(out[i].find('s', {'id':f'vn{i+1}'}).getText())
      if len(v.split()) <= 12:
        en.append(e)
        vn.append(v)
        # print(e, v)

In [13]:
freq = pd.read_html("https://1000mostcommonwords.com/1000-most-common-vietnamese-words/", header=0)[0]
freq[20:30]

,Number,Vietnamese,in English
20,21,nhưng,but
21,22,những gì,what
22,23,một số,some
23,24,là,is
24,25,nó,it
25,26,anh,you
26,27,hoặc,or
27,28,có,had
28,29,các,the
29,30,của,of


In [26]:
# TODO: unfinished attempt to split the words
import numpy as np
freq1 = freq.copy()
temp = freq1.Vietnamese.str.split(' ')
freq1 = freq1.reindex(freq1.index.repeat(temp.apply(len)))
freq1.Vietnamese = np.hstack(temp)
# freq1.Vietnamese = freq1.Vietnamese.str.split(" ")
freq1[20:30]

# data = data.reindex(data.index.repeat(temp.apply(len)))
# data['new_Seatblocks'] = np.hstack(temp)

,Number,Vietnamese,in English
20,21,nhưng,but
21,22,những,what
21,22,gì,what
22,23,một,some
22,23,số,some
23,24,là,is
24,25,nó,it
25,26,anh,you
26,27,hoặc,or
27,28,có,had


In [27]:
len(freq1)

1509

In [ ]:
len(en)
len(vn)

9884

In [ ]:
vn[-70]

'Thực tiễn này đẩy chúng ta đến đâu '

In [ ]:
out[50]

<spair id="51">
<s id="en51">We had a 3/4 week of it</s>
<s id="vn51">Chúng ta đã có 3/4 tuần</s>
<a id="ev51">1-1,2;2-3,4;4-5;5-6;</a>
</spair>

In [ ]:
pd.read_xml("N0001.sgml")

,title,corpus,author,citation,release,spair
0,What is a Fenqing ?,EVBCorpus,"Quoc-Hung Ngo, Werner Winiwarter","Quoc-Hung Ngo, Werner Winiwarter, 2012. ""Build...",22/10/2013,NaN
1,None,None,None,None,None,NaN
